### install tutorial dataset
https://www.kaggle.com/datasets/deepakat002/roundabout-aerial-images-yolo-data/data


In [1]:
# setting

import os
HOME = os.getcwd()
print(HOME)
from IPython.display import Image
import glob
from IPython.display import Image, display
import random
import yaml

import shutil
!pip install tqdm --upgrade
from tqdm.notebook import tqdm

/content


In [2]:
# gpu
!nvidia-smi

Tue Sep  3 02:23:13 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
# ultralytics
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.0/872.0 kB 28.2 MB/s eta 0:00:00


In [4]:
# check ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.86 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.8/78.2 GB disk)


In [23]:
# drive mount  -> Automatically run the cell again whenever the drive is updated.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# train_test_split, test data directories

train_path_img = "/content/yolo_data/train/images/"
train_path_label = "/content/yolo_data/train/labels/"
val_path_img = "/content/yolo_data/valid/images/"
val_path_label = "/content/yolo_data/valid/labels/"
test_path = "/content/yolo_data/test/images/"
test_path_label = "/content/yolo_data/test/labels/"

In [9]:
# split dataset

def createdataYaml(train_path,val_path,class_names =[],outdir = "/content/"):
  data = {"train":train_path,
          "val": val_path,
          "nc": len(class_names),
          "names": class_names
  }
  def write_yaml_to_file(py_obj,filename):
    with open(f"{filename}.yaml", "w",)as f:
      yaml.dump(py_obj,f,sort_keys=False)
    print(f"written to file {filename}.yaml successfully")

  datasetyamloutdir = outdir+"dataset"
  write_yaml_to_file(data,datasetyamloutdir)

def train_test_split(path,neg_path=None, split = 0.2):


  files = list(set([name[:-4] for name in os.listdir(path)]))

  random.seed(42)
  random.shuffle(files)

  test_size = int(len(files)* split)
  train_size = len(files) - test_size

  os.makedirs(train_path_img, exist_ok = True)
  os.makedirs(train_path_label, exist_ok = True)
  os.makedirs(val_path_img, exist_ok = True)
  os.makedirs(val_path_label, exist_ok = True)


  for filex in tqdm(files[:train_size]):
    if filex =="classes":
      continue
    shutil.copy2(path + filex + ".jpg", f"{train_path_img}/" + filex + ".jpg")
    shutil.copy2(path + filex + ".txt", f"{train_path_label}/" + filex + ".txt")


  if neg_path:
    neg_images = list(set([name[:-4] for name in os.listdir(neg_path)]))
    for filex in tqdm(neg_images):
      shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/"+filex+ ".jpg")

  for filex in tqdm(files[train_size:]):
    if filex =="classes":
      continue
    shutil.copy2(path + filex + ".jpg", f"{val_path_img}/" + filex + ".jpg")
    shutil.copy2(path + filex + ".txt", f"{val_path_label}/" + filex + ".txt")

train_test_split("/content/drive/MyDrive/yolov10/data/")

createdataYaml(train_path=train_path_img,val_path=val_path_img,class_names=["car","cycle","bus","truck","van"],outdir = "/content/")

  0%|          | 0/782 [00:00<?, ?it/s]

  0%|          | 0/195 [00:00<?, ?it/s]

written to file /content/dataset.yaml successfully


In [10]:
# install yolov10
!pip install -q git+https://github.com/THU-MIG/yolov10.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [11]:
# weights download

!wget -P {HOME}/weights -q https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10m.pt

In [12]:
# find path of weights
!ls -la {HOME}/weights

total 65368
drwxr-xr-x 2 root root     4096 Sep  3 02:27 .
drwxr-xr-x 1 root root     4096 Sep  3 02:27 ..
-rw-r--r-- 1 root root 66924131 May 23 09:38 yolov10m.pt


In [13]:
# run train -> can change parameters
%cd {HOME}
!yolo task=detect mode=train model ={HOME}/weights/yolov10m.pt data =/content/dataset.yaml epochs =50 imgsz= 640 batch = 8 \
--project=/content/drive/MyDrive/yolov10/training_backup name=traffic2 --optimizer=Adam

/content
WARNING ⚠️ argument '--project=/content/drive/MyDrive/yolov10/training_backup' does not require leading dashes '--', updating to 'project=/content/drive/MyDrive/yolov10/training_backup'.
WARNING ⚠️ argument '--optimizer=Adam' does not require leading dashes '--', updating to 'optimizer=Adam'.
/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:733: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `to

In [24]:
# inference with trained model:

#cd {HOME}

# !yolo task=detect mode=predict model = /content/drive/MyDrive/yolov10/training_backup/traffic2/weights/best.pt conf =0.55 source = /content/drive/MyDrive/yolov10/test/images
!yolo task=detect mode=predict model = /content/drive/MyDrive/yolov10/training_backup/layout_detection.pt conf =0.55 source = /content/drive/MyDrive/yolov10/test/images

/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:733: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")
Ultralytics YO

In [18]:
pwd

'/content'

In [25]:
# result files move to my dir
!cp -r /content/runs/detect/predict /content/drive/MyDrive/yolov10/output/images